In [6]:
import pandas as pd
import numpy as np
import geoip2.database

In [7]:
merged_df = pd.read_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/feb5-12_merged.csv')
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)
merged_df = merged_df.dropna()

In [8]:
reader = geoip2.database.Reader('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/GeoLite2-City.mmdb')
# Build location info
locations = []

for ip in merged_df['Src IP']:
    try:
        response = reader.city(ip)
        country = response.country.name or "Unknown Country"
        city = response.city.name or "Unknown City"

        location_str = f"{country}"
    except Exception as e:
        location_str = f"Error: {e}"

    locations.append(location_str)

# Add to DataFrame
merged_df['Location'] = locations

In [9]:
'''
90% of our data is represented in 10 columns, as such we are choosing to limit our model to those countries
'''
merged_df = merged_df[(merged_df['Location'] == 'Iran') | (merged_df['Location'] == 'United States') | (merged_df['Location'] == 'Hong Kong') | (merged_df['Location'] == 'China') | (merged_df['Location'] == 'Germany') | (merged_df['Location'] == 'Japan') | (merged_df['Location'] == 'Italy') | (merged_df['Location'] == 'France') | (merged_df['Location'] == 'Singapore') | (merged_df['Location'] == 'United Kingdom')]
merged_labels = merged_df
merged_labels['Labels'] = merged_labels['Label'].map({'ddospot':1,'cowrie':2,'adbhoney':3,'log4pot':4,'ciscoasa':5,'elasticpot':6,'mailoney':7})

In [10]:
threshold = 0.98
dropped = []
for col in merged_df.columns:
    top_freq = merged_df[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_freq > threshold:
        dropped.append(col)
        merged_df = merged_df.drop(columns=col)
merged_df.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Timestamp', 'Flow Duration',
       'Total Fwd Packet', 'Total Bwd packets', 'Total Length of Fwd Packet',
       'Total Length of Bwd Packet', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min',
       'Packet Length Max', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'Down/Up Ratio', 'Average Packet Size',
       'Fwd Segment Size Avg', 'Bwd Segment Size Avg', 'Bwd Bytes/Bulk Avg',
       'Bwd Packet/Bulk 

In [11]:
merged_df['Country'] = merged_df['Location'].map({'Iran':1,'United States':2,'Hong Kong':3,'China':4,'Germany':5,'Japan':6,'Italy':7,'France':8,'Singapore':9,'United Kingdom':10})

In [12]:
merged_df = merged_df.drop(columns=['Flow ID','Src IP','Src Port','Timestamp','day','Fwd Act Data Pkts', 'Fwd Header Length', 
                                    'Fwd Packet Length Mean','Total Length of Bwd Packet', 'Bwd Header Length', 'Bwd Segment Size Avg', 
                                    'Bwd Bytes/Bulk Avg','Bwd IAT Min','Bwd IAT Max', 'Bwd IAT Total','Packet Length Max','Packet Length Std',
                                    'Packet Length Variance','Average Packet Size','Active Min','Active Max','Idle Min', 'Idle Std','Location'])
merged_noBwd = merged_df.drop(columns=['Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std','Bwd Packets/s','Bwd Packet/Bulk Avg',
       'Bwd Bulk Rate Avg','Subflow Bwd Bytes'])
merged_labels = merged_labels.drop(columns=['Dst IP','Dst Port','Protocol',
 'Fwd Packet Length Std',
 'Bwd IAT Std',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'FIN Flag Count',
 'SYN Flag Count',
 'RST Flag Count',
 'PSH Flag Count',
 'ACK Flag Count',
 'URG Flag Count',
 'CWR Flag Count',
 'ECE Flag Count',
 'Fwd Bytes/Bulk Avg',
 'Fwd Packet/Bulk Avg',
 'Fwd Bulk Rate Avg',
 'Subflow Bwd Packets',
 'FWD Init Win Bytes',
 'Bwd Init Win Bytes',
 'Fwd Seg Size Min',
 'Active Std',
 'Label'])
merged_labels = merged_labels.drop(columns=['Flow ID','Src IP','Src Port','Timestamp','day','Fwd Act Data Pkts', 'Fwd Header Length', 
                                            'Fwd Packet Length Mean','Total Length of Bwd Packet', 'Bwd Header Length', 'Bwd Segment Size Avg', 
                                            'Bwd Bytes/Bulk Avg','Bwd IAT Min','Bwd IAT Max', 'Bwd IAT Total','Packet Length Max','Packet Length Std',
                                            'Packet Length Variance','Average Packet Size','Active Min','Active Max','Idle Min', 'Idle Std','Location'])

In [13]:
merged_df.to_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/Clean4Model')
merged_labels.to_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/Clean4Labels')
merged_noBwd.to_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/CleanNoBwd')